In [1]:
from hypersurface import *
import sympy as sp
from pprint import pprint
from scipy.optimize import minimize
from sympy.utilities.iterables import flatten
import math

In [2]:
z0, z1, z2, z3, z4= sp.symbols('z0, z1, z2, z3, z4')
Z = [z0,z1,z2,z3,z4]
f = z0**5 + z1**5 + z2**5 + z3**5 + z4**5
HS = Hypersurface(Z, f, 100)

In [3]:
HS.set_k(1)
ns = HS.n_sections

In [4]:
# holomorphic=True by default
factor = (HS.integrate(lambda patch, point: patch.omega_omegabar(point), numerical=True) / \
          HS.integrate(lambda patch, point: patch.num_FS_volume_form('identity', point), numerical=True)).real

The factor can be a build-in attribute and calculated patchwise. Design a function HS.calculate_factor() and call it before num_eta()

In [5]:
def flat_to_matrix(g_flat):
    g_real = g_flat[:len(g_flat)//2].reshape(ns, ns)
    g_imag = g_flat[len(g_flat)//2:].reshape(ns, ns)
    g_matrix = g_real + 1j * g_imag
    return g_matrix

In [6]:
def integration(g_flat):    
    g = flat_to_matrix(g_flat)
    h = g*np.conj(g.transpose())
    integration = HS.integrate(lambda patch, point, h_matrix=h: 
                                np.absolute(patch.num_eta(h_matrix, point)*factor - 1).real, 
                                holomorphic=True, numerical=True)
    return integration

The first matrix is the real part and second is the imaginary part.

In [33]:
#g0 = np.concatenate((np.ones(ns**2), np.ones(ns**2)),axis=None)*math.sqrt(1/2)
g0 = np.concatenate((np.identity(ns), np.identity(ns)), axis=None)*math.sqrt(1/2)
noise_real = 2 * np.random.rand(ns,ns) - 1
noise_imag = 2 * np.random.rand(ns,ns) - 1
noise = np.concatenate((noise_real, noise_imag), axis=None)*0.1
g0 = g0 + noise

In [ ]:
res = minimize(integration, g0, method='L-BFGS-B')

In [ ]:
g_minimal = res.x

In [ ]:
g = flat_to_matrix(g_minimal)
h_minimal = g*np.conj(g.transpose())

In [23]:
HS.integrate(lambda patch, point, h_matrix='identity': 
                np.absolute(patch.num_eta(h_matrix, point)*factor - 1), 
                holomorphic=True, numerical=True)

(0.4726107430536371+1.6481306624358407e-19j)

In [ ]:
HS.integrate(lambda patch, point, h_matrix=h_minimal: 
                np.absolute(patch.num_eta(h_matrix, point)*factor - 1), 
                holomorphic=True, numerical=True)